# Modelling the effects of individual variation and superspreading on poisson noise branching process epidemic model with StochasticNegBinBranchPro

The first part of the notebook includes a forward simulation of the incidence numbers for an example branching process model with both local and imported cases and negative binomial distributed noise with individual variation in the reproduction numbers for different dispersion values. The underlying population reproduction number profile is assumed to be a step function with only one jump. The results are displayed as a barplot and sliders.

The second part of the notebook focuses on the changes in the inferred $R_t$ (uses Poisson-distributed inference functionalities) profiles subject to changes in the overdispesion of the negative binomial branching process model with with individual variation in the reproduction numbers. This is shown via sliders. In doing so, we will illustrate the impact of noise on the quality of parameter inference.

In [1]:
# Import libraries
import numpy as np
import math
import branchpro
import scipy.stats
import matplotlib
from branchpro.apps import ReproductionNumberPlot
import plotly.graph_objects as go
from matplotlib import pyplot as plt
import os
import pandas as pd
import datetime

num_timepoints = 30 # number of days for incidence data

# Build the imported cases
ic_mean = 70
imported_times = np.arange(1,(num_timepoints+1))
imported_cases = scipy.stats.poisson.rvs(ic_mean, size=num_timepoints)

## Parameterize example negative binomial noise branching process model with local and imported cases

In [2]:
# Build the serial interval w_s
ws_mean = 2.6
ws_var = 1.5**2
theta = ws_var / ws_mean
k = ws_mean / theta
 
w_dist = scipy.stats.gamma(k, scale=theta)
disc_w = w_dist.pdf(np.arange(num_timepoints))

In [3]:
# Construct LocImpBranchProModel object
epsilon = 1
phi = 0.5

initial_r = 3
serial_interval = disc_w
m = branchpro.StochasticLocImpNegBinBranchProModel(initial_r, serial_interval, epsilon, phi)

new_rs = [1.5, 0.5]          # sequence of R_0 numbers
start_times = [0, 15]      # days at which each R_0 period begins
m.set_r_profile(new_rs, start_times)
parameters = 100 # initial number of cases
times = np.arange(num_timepoints)

m.set_imported_cases(imported_times, imported_cases)
locally_infected_cases = m.simulate(parameters, times)

print(locally_infected_cases)

[100.   0.  31.  66.  97. 144. 174. 229. 303. 328. 394. 482. 580. 666.
 799. 948. 351. 440. 396. 304. 278. 255. 203. 206. 167. 153. 142. 119.
 144. 122.]


## Plot local incidence numbers and imported cases for the forward simulation

In [4]:
# Plot (bar chart cases each day)
fig = go.Figure()

# Plot of incidences
fig.add_trace(
    go.Bar(
        x=times,
        y=locally_infected_cases,
        name='Incidences'
    )
)

fig.add_trace(
    go.Bar(
        x=times,
        y=imported_cases,
        name='Imported Incidences'
    )
)

# Add axis labels
fig.update_layout(
    xaxis_title='Time (days)',
    yaxis_title='New cases'
)

fig.show()


## Sensitivity analysis

### 1. Varying overdispersion

In [5]:
# Slider (overdispersion)
fig = go.Figure()

# Add traces, one for each slider step
for phi in np.arange(0.01, 10, 0.01):
    m.set_overdispersion(phi)

    cases = m.simulate(parameters, times)
    
    fig.add_trace(
        go.Bar(
            x=times,
            y=cases,
            name='Incidences'
        ))
        
    fig.add_trace(
    go.Bar(
        x=times,
        y=imported_cases,
        name='Imported Incidences'
    ))

# Make 10th trace visible
fig.data[10].visible = True

# Create and add slider
steps = []
for i in range(int(len(fig.data)/2)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Slider switched to overdispersion: " + str(np.arange(0.01, 1, 0.01)[i])}],  # layout attribute
        label = str(np.arange(0.01, 1, 0.01)[i])
    )
    step["args"][0]["visible"][2 * i] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][1 + 2 * i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Overdispersion: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

# Add axis labels
fig.update_layout(
    xaxis_title='Time (days)',
    yaxis_title='New cases'
)

fig.show()

## Compute the posterior distribution using class method for different overdispersion values

In [6]:
# Same inference, but using the LocImpBranchProPosterior
epsilon = 1
tau = 6
R_t_start = tau+1
a = 1
b = 1/5

L1 = len(np.arange(R_t_start, start_times[1]+1))
L2 = len(np.arange(start_times[1]+1, num_timepoints))

ground_truth = pd.DataFrame({
    'Time Points': np.arange(R_t_start, num_timepoints),
    'R_t': L1*[1.5] + L2*[0.5]
})

In [7]:
# Slider (overdispersion)
fig = ReproductionNumberPlot()

# Add traces, one for each slider step
for phi in np.arange(0.01, 1, 0.01):
    m.set_overdispersion(phi)

    locally_infected_cases = m.simulate(parameters, times)

    # Transform our incidence data into pandas dataframes
    inc_data = pd.DataFrame(
        {
            'Time': np.arange(num_timepoints),
            'Incidence Number': locally_infected_cases
        }
    )

    imported_inc_data = pd.DataFrame(
        {
            'Time': np.arange(num_timepoints),
            'Incidence Number': imported_cases
        }
    )

    inference = branchpro.LocImpBranchProPosterior(
        inc_data=inc_data,
        imported_inc_data=imported_inc_data,
        epsilon=epsilon,
        daily_serial_interval=serial_interval,
        alpha=a,
        beta=1/b)

    inference.run_inference(tau=tau)
    intervals = inference.get_intervals(central_prob=.95)

    fig.add_ground_truth_rt(ground_truth)
    fig.add_interval_rt(intervals)

# Make 10th trace visible
fig.figure.data[10].visible = True

# Create and add slider
steps = []
for i in range(int(len(fig.figure.data)/3)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.figure.data)},
              {"title": "Slider switched to overdispersion: " + str(np.arange(0.01, 1, 0.01)[i])}],  # layout attribute
        label = str(np.arange(0.01, 1, 0.01)[i])
    )
    step["args"][0]["visible"][3 * i] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][1 + 3 * i] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][2 + 3 * i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Overdispersion: "},
    pad={"t": 50},
    steps=steps
)]

fig.figure.update_layout(
    sliders=sliders
)

fig.update_labels(time_label='Time (Day)', r_label='R_t')

fig.show_figure()

/Users/ioaros/Desktop/Software Project/branchpro/branchpro/apps/_reproduction_number_plot.py:27: UserWarning:

Labels do not match. They will be updated.

/Users/ioaros/Desktop/Software Project/branchpro/branchpro/apps/_reproduction_number_plot.py:27: UserWarning:

Labels do not match. They will be updated.

/Users/ioaros/Desktop/Software Project/branchpro/branchpro/apps/_reproduction_number_plot.py:27: UserWarning:

Labels do not match. They will be updated.

/Users/ioaros/Desktop/Software Project/branchpro/branchpro/apps/_reproduction_number_plot.py:27: UserWarning:

Labels do not match. They will be updated.

/Users/ioaros/Desktop/Software Project/branchpro/branchpro/apps/_reproduction_number_plot.py:27: UserWarning:

Labels do not match. They will be updated.

/Users/ioaros/Desktop/Software Project/branchpro/branchpro/apps/_reproduction_number_plot.py:27: UserWarning:

Labels do not match. They will be updated.

/Users/ioaros/Desktop/Software Project/branchpro/branchpro/apps/_repro